# Running a raffle with Marlowe

The raffle contract receives a deposit of a prize and then an oracle randomly selects the winner. The contract would be too large to fit on the blockchain because of the hundreds of possible winners, so Marlowe's merkleization capability is used to break up the contract into manageable pieces. The pieces are "glued together" using Marlowe's `Notify` block, so that the contract can walk a hierarchy of possible winners down to the actual winner.

## Prerequisites

### Define the prizes

#### First prize

We need to know the policy ID and token name for the NFT serves as the Marlowe role token for redeeming the first prize. There is no requirement for this token beyond that we know its policy ID and token name.

In [ ]:
# Replace with actual policy and name!
PRIZE1_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE1_NAME=1stPrize

Now create a data file containing this information.

In [ ]:
cat << EOI > first-prize.token
[
    [
        "$PRIZE1_POLICY",
        "$PRIZE1_NAME"
    ]
]
EOI
cat first-prize.token

#### Second prize

Now repeat the same steps for the second prize.

In [ ]:
# Replace with actual policy and name!
PRIZE2_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE2_NAME=2ndPrize

Now create a data file containing this information.

In [ ]:
cat << EOI > second-prize.token
[
    [
        "$PRIZE2_POLICY",
        "$PRIZE2_NAME"
    ]
]
EOI
cat second-prize.token

#### Third prize

Now repeat the same steps for the third prize.

In [ ]:
# Replace with actual policy and name!
PRIZE3_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE3_NAME=3rdPrize

Now create a data file containing this information.

In [ ]:
cat << EOI > third-prize.token
[
    [
        "$PRIZE3_POLICY",
        "$PRIZE3_NAME"
    ]
]
EOI
cat third-prize.token

### Ensure that the three prize tokens reside at the sponsor address

Send the three prize tokens (the Marlowe role tokens) to the sponsor address.

In [ ]:
SPONSOR_SKEY=sponsor.skey
SPONSOR_ADDR=$(cat sponsor.mainnet.address)
echo "SPONSOR_ADDR = $SPONSOR_ADDR"

After the tokens have been sent, we can query to see them.

In [ ]:
cardano-cli query utxo --mainnet --address $SPONSOR_ADDR

### Download the executables to run the raffle.

The script residing at https://github.com/input-output-hk/real-world-marlowe/tree/main/archives/raffle are compiled to executables that should work on any recent Linux machine.

In [ ]:
curl -sS http://dl.marlowestat.org/raffle-bin.tar -o raffle-bin.tar

Unpack the executable files.

In [ ]:
tar xvf raffle-bin.tar

Put the executables at the front of the search path.

In [ ]:
export PATH=$PWD/bin:$PATH

### Create the configuration files

Each prize has its own configuration file.

#### Configuration for first prize's raffle

In [ ]:
yaml2json << EOI > first-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 3d         # Deadline for depositing the prize token.
  selectWinner: 3d    # Deadline for selecting the winner.
  payout: 3d          # Deadline for paying the winner.
runtimeURI:
  proxy_host: 34.70.189.123  # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_HOST if a backup is needed.
  proxy_port: 3700           # The port number for Marlowe Runtime's proxy service. Use $MARLOWE_RT_PORT for the backup.
  web_host: 34.70.189.123    # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_WEBSERVER_HOST if a backup is needed.
  web_port: 3780             # The port number for Marlowe Runtime's web server. Use $MARLOWE_RT_WEBSERVER_PORT for the backup.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: first-prize.contract
state: first-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat first-prize.config

#### Configuration for second prize's raffle

In [ ]:
yaml2json << EOI > second-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 3d         # Deadline for depositing the prize token.
  selectWinner: 3d    # Deadline for selecting the winner.
  payout: 3d          # Deadline for paying the winner.
runtimeURI:
  proxy_host: 34.70.189.123  # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_HOST if a backup is needed.
  proxy_port: 3700           # The port number for Marlowe Runtime's proxy service. Use $MARLOWE_RT_PORT for the backup.
  web_host: 34.70.189.123    # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_WEBSERVER_HOST if a backup is needed.
  web_port: 3780             # The port number for Marlowe Runtime's web server. Use $MARLOWE_RT_WEBSERVER_PORT for the backup.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: second-prize.contract
state: second-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat second-prize.config

#### Configuration for third prize's raffle

In [ ]:
yaml2json << EOI > third-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 3d         # Deadline for depositing the prize token.
  selectWinner: 3d    # Deadline for selecting the winner.
  payout: 3d          # Deadline for paying the winner.
runtimeURI:
  proxy_host: 34.70.189.123  # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_HOST if a backup is needed.
  proxy_port: 3700           # The port number for Marlowe Runtime's proxy service. Use $MARLOWE_RT_PORT for the backup.
  web_host: 34.70.189.123    # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_WEBSERVER_HOST if a backup is needed.
  web_port: 3780             # The port number for Marlowe Runtime's web server. Use $MARLOWE_RT_WEBSERVER_PORT for the backup.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: third-prize.contract
state: third-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat third-prize.config

## Description of the raffle

The raffle contract receives a deposit of a prize and then an oracle randomly selects the winner. The contract would be too large to fit on the blockchain because of the hundreds of possible winners, so Marlowe's merkleization capability is used to break up the contract into manageable pieces. The pieces are "glued together" using Marlowe's `Notify` block, so that the contract can walk a hierarchy of possible winners down to the actual winner.

Below is a diagram of the raffle contract, showing the transactions as arrows:
1.  Create the contract.
2.  Deposit the prize token.
3.  The oracle randomly chooses a winner.
4.  The first notify branches five-fold according the the random number chosen by the oracle.
    -   The *first* branch is taken if the winner is among the *first* group of 125 addresses.
    -   The *second* branch is taken if the winner is among the *second* group of 125 addresses.
    -   The *third* branch is taken if the winner is among the *third* group of 125 addresses.
    -   The *fourth* branch is taken if the winner is among the *fourth* group of 125 addresses.
    -   The *fifth* branch is taken if the winner is among the *fifth* group of 125 addresses.
5.  The second notify branches five-fold according the the random number chosen by the oracle.
    -   The first branch is taken if the winner is among the first group of 25 addresses within the previously selected 125 addresses.
    -   Etc.
6.  The third notify branches five-fold according the the random number chosen by the oracle.
    -   The first branch is taken if the winner is among the first group of 5 addresses within the previously selected 25 addresses.
    -   Etc.
7.  The fourth notify branches five-fold according the the random number chosen by the oracle.
    -   The first branch is taken if the winner is among the first of the previously selected 5 addresses, and the prize is paid to that address.
    -   Etc.
    
![Hierarchy of transactions in the raffle contract.](hierarchy.png)

## Run the raffles

### Run the raffle for the first prize

First view the start of the file with the addresses of the raffle ticket holders.

In [ ]:
head first-prize.addresses

Now build and submit the transaction that initializes the raffle.

In [ ]:
InitializeRaffle first-prize.config first-prize.addresses first-prize.token

Record the contract ID that was printed.

In [ ]:
# IMPORTANT: Record the contract ID in the variable below!
PRIZE1_CONTRACT=

Run the raffle itself, which consists of several transactions:
1. Deposit the prize token.
2. Have the oracle retrieve a random number from https://www.random.org/.
3. Execute several Marlowe notify transactions to locate the winner.
4. Send the prize token to the winner.

In [ ]:
ExecuteRaffle first-prize.config first-prize.addresses first-prize.token $PRIZE1_CONTRACT

View the contract on MarloweScan.

In [ ]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE1_CONTRACT%%#1}%231"

Record the random number from that the oracle choose, listed in the output above.

In [ ]:
# IMPORTANT: Record the number the oracle answered with in the variable below!
PRIZE1_CHOICE=

Compute the address of the winner and view their prize on an explorer.

In [ ]:
PRIZE1_WINNER=$(jq -r ".[$PRIZE1_CHOICE].payment_address" first-prize.addresses)
echo "PRIZE1_WINNER = $PRIZE1_WINNER"
echo "https://cardanoscan.io/address/$PRIZE1_WINNER"

### Run the raffle for the second prize

Now create the file for those eligible for the second prize by deleting the entry for the first prize.

In [ ]:
jq "del(.[$PRIZE1_CHOICE])" first-prize.addresses > second-prize.addresses

Now build and submit the transaction that initializes the raffle.

In [ ]:
InitializeRaffle second-prize.config second-prize.addresses second-prize.token

Record the contract ID that was printed.

In [ ]:
# IMPORTANT: Record the contract ID in the variable below!
PRIZE2_CONTRACT=

Run the raffle itself, which consists of several transactions.

In [ ]:
ExecuteRaffle second-prize.config second-prize.addresses second-prize.token $PRIZE2_CONTRACT

View the contract on MarloweScan.

In [ ]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE2_CONTRACT%%#1}%231"

Record the random number from that the oracle choose, listed in the output above.

In [ ]:
# IMPORTANT: Record the number the oracle answered with in the variable below!
PRIZE2_CHOICE=

Compute the address of the winner and view their prize on an explorer.

In [ ]:
PRIZE2_WINNER=$(jq -r ".[$PRIZE2_CHOICE].payment_address" second-prize.addresses)
echo "PRIZE2_WINNER = $PRIZE2_WINNER"
echo "https://cardanoscan.io/address/$PRIZE2_WINNER"

### Run the raffle for the thrid prize

Now create the file for those eligible for the second prize by deleting the entry for the second prize.

In [ ]:
jq "del(.[$PRIZE2_CHOICE])" second-prize.addresses > third-prize.addresses

Now build and submit the transaction that initializes the raffle.

In [ ]:
InitializeRaffle third-prize.config third-prize.addresses third-prize.token

Record the contract ID that was printed.

In [ ]:
# IMPORTANT: Record the contract ID in the variable below!
PRIZE3_CONTRACT=

Run the raffle itself, which consists of several transactions.

In [ ]:
ExecuteRaffle third-prize.config third-prize.addresses third-prize.token $PRIZE3_CONTRACT

View the contract on MarloweScan.

In [ ]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE3_CONTRACT%%#1}%231"

Record the random number from that the oracle choose, listed in the output above.

In [ ]:
# IMPORTANT: Record the number the oracle answered with in the variable below!
PRIZE3_CHOICE=

Compute the address of the winner and view their prize on an explorer.

In [ ]:
PRIZE3_WINNER=$(jq -r ".[$PRIZE3_CHOICE].payment_address" third-prize.addresses)
echo "PRIZE3_WINNER = $PRIZE3_WINNER"
echo "https://cardanoscan.io/address/$PRIZE3_WINNER"